In [1]:
import pandas as pd
from utils import *
import os
import fitz
from pdf2image import convert_from_path
import numpy as np

#### Step 1: Change Working Directory

In [2]:
os.chdir('/Users/sooyeonkim/Desktop/TenStrands')

In [3]:
master = pd.read_excel('District Level Master Spreadsheet-UCB Policy Analysis - BP_AR 3514.xlsx')

In [4]:
master['BP3514'] = master['District Name'].str.replace(' ', '_') + '_' + master['County'].str.replace(' ', '_') + '_BP3514.pdf'
master['AR3514'] = master['District Name'].str.replace(' ', '_') + '_' + master['County'].str.replace(' ', '_') + '_AR3514.pdf'
master.to_excel('District Level Master Spreadsheet-UCB Policy Analysis - BP_AR 3514.xlsx', index=False)

In [5]:
# base paths
base_path_ar = "/Users/sooyeonkim/Desktop/TenStrands/AR_pdf/"
base_path_bp = "/Users/sooyeonkim/Desktop/TenStrands/BP_pdf/"

# constructing full paths
master['AR3514: Path to PDF'] = base_path_ar + master['AR3514'].astype(str)
master['BP3514: Path to PDF'] = base_path_bp + master['BP3514'].astype(str)

#### Step 2: Load Data

In [6]:
import pdfplumber
import pandas as pd
import os

# specify the directory containing the PDF files
pdf_directory = "/Users/sooyeonkim/Desktop/TenStrands/pdf_files"

In [7]:
# load  dataset that contains paths to PDFs
df = master

# initialize columns to store PDF text
df['BP3514: PDF Text'] = np.nan
df['AR3514: PDF Text'] = np.nan

# set the columns to object type explicitly to store strings
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].astype('object')
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].astype('object')

In [8]:
def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file."""
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ''.join([page.extract_text() or '' for page in pdf.pages])
        return text
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")
        return None  

In [9]:
for index, row in df.iterrows():
    if pd.notna(row['AR3514: Path to PDF']):
        ar_text = extract_text_from_pdf(row['AR3514: Path to PDF'])
        df.at[index, 'AR3514: PDF Text'] = ar_text

    if pd.notna(row['BP3514: Path to PDF']):
        bp_text = extract_text_from_pdf(row['BP3514: Path to PDF'])
        df.at[index, 'BP3514: PDF Text'] = bp_text

Failed to process /Users/sooyeonkim/Desktop/TenStrands/BP_pdf/Albany_City_Unified_Alameda_BP3514.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/BP_pdf/Albany_City_Unified_Alameda_BP3514.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/AR_pdf/Fremont_Unified_Alameda_AR3514.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/AR_pdf/Fremont_Unified_Alameda_AR3514.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/BP_pdf/Fremont_Unified_Alameda_BP3514.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/BP_pdf/Fremont_Unified_Alameda_BP3514.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/AR_pdf/New_Haven_Unified_Alameda_AR3514.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/AR_pdf/New_Haven_Unified_Alameda_AR3514.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/AR_pdf/San_Lorenzo_Unified_Alameda_AR3514.pdf: [Errno 2] No such file

In [10]:
df[df['BP3514: PDF Text'].isna() & (df['BP: 3514 Environmental Safety'] == 1)]['CDS Code']

4       1611760000000
56      9619290000000
64     10621170000000
73     10624140000000
79     10739990000000
83     10767780000000
108    13630810000000
121    14766870000000
142    15637500000000
174    18641390000000
221    19648810000000
266    22655320000000
337    30666130000000
385    34752830000000
400    36677020000000
403    36677770000000
410    36678680000000
413    36679180000000
415    36679590000000
425    37680070000000
428    37680560000000
434    37681300000000
458    38684780000000
461    39685690000000
464    39685930000000
508    42692600000000
518    43694270000000
533    43696410000000
539    44697320000000
589    49707220000000
633    51714230000000
650    54718600000000
654    54719930000000
662    54721990000000
663    54722070000000
674    55723480000000
685    56725120000000
686    56725200000000
694    56738740000000
Name: CDS Code, dtype: int64

In [15]:
df[df['AR3514: PDF Text'].isna() & (df['AR: 3514 Environmental Safety Regulation'] == 1)]['CDS Code']

4       1611760000000
30      6616060000000
37      7617050000000
57      9619520000000
79     10739990000000
80     10752340000000
100    12630320000000
110    13631070000000
174    18641390000000
215    19648080000000
221    19648810000000
255    20764140000000
257    21653910000000
274    23739160000000
300    27660680000000
382    34674390000000
385    34752830000000
400    36677020000000
413    36679180000000
417    36738900000000
425    37680070000000
428    37680560000000
434    37681300000000
444    37683380000000
446    37683610000000
464    39685930000000
467    39686760000000
503    42692030000000
518    43694270000000
531    43696170000000
589    49707220000000
654    54719930000000
663    54722070000000
674    55723480000000
Name: CDS Code, dtype: int64

In [16]:
import re

def clean_text(text):
    """Remove or replace illegal XML characters from the text."""
    if text is not None and isinstance(text, str):
        # XML 1.0 valid characters:
        illegal_xml_chars = re.compile(u'[\u0000-\u0008\u000B\u000C\u000E-\u001F\uD800-\uDFFF\uFFFE\uFFFF]')
        return illegal_xml_chars.sub('', text)
    return text


In [17]:
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].apply(clean_text)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].apply(clean_text)


In [18]:
def clean_text(text):
    if pd.isna(text):  # check if the text is NaN
        return text  # return as is or consider "return ''" to replace NaN with empty string
        
    # removing line breaks and excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # removing URLs 
    text = re.sub(r'https?://\S+', '', text)
        
    # lowercase the text
    text = text.lower()
    
    return text

# applying cleaning function to text columns
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].apply(clean_text)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].apply(clean_text)

In [ ]:
df.to_excel("BP_AR_3514.xlsx")
df.to_csv("BP_AR_3514.csv")